## install additional packages

In [10]:
!pip install deap k3d mpld3 > /dev/null
%matplotlib ipympl
import mpld3
mpld3.enable_notebook()

# Configuration

In [2]:
from pathlib import Path
import json

class Config(object):
    def __init__(self, algorithmType, w, c1, c2, iterations, target_error, n_particles, arguments_dimensions, d_min, d_max, fitness):
        self.algorithmType = algorithmType
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.iterations = iterations
        self.target_error = target_error
        self.n_particles = n_particles
        self.arguments_dimensions = arguments_dimensions
        self.d_min = d_min
        self.d_max = d_max
        self.d_max = d_max
        self.fitness = fitness


def as_config(dct):
    return Config(
        dct['algorithmType'],
        dct['w'],
        dct['c1'],
        dct['c2'],
        dct['iterations'], 
        dct['target_error'], 
        dct['n_particles'],
        dct['arguments_dimensions'],
        dct['d_min'], 
        dct['d_max'],
        dct['fitness']
        )

# Genetic algorithm using deap

In [3]:
import random
import numpy
import inspect
from deap import algorithms, base, creator, tools

# translates into tuple
def gaFitness(individual):
    return fitness(individual),

def cxTwoPointCopy(ind1, ind2):
    size = len(ind1)
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2].copy(), ind1[cxpoint1:cxpoint2].copy()
        
    return ind1, ind2

def genetic():
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", numpy.ndarray, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_bool", lambda: random.random()*cfg.d_max)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=cfg.arguments_dimensions)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", gaFitness)
    toolbox.register("mate", cxTwoPointCopy)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    pop = toolbox.population(n=cfg.n_particles)
    hof = tools.HallOfFame(1, similar=numpy.array_equal)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=cfg.iterations, stats=stats, halloffame=hof)

    return pop, stats, hof

# particle swarm algorithm classes

In [4]:
from random import getrandbits
import numpy as np

class Particle():
    def __init__(self):
        self.position = (cfg.d_max - cfg.d_min) * np.random.random_sample(cfg.arguments_dimensions)+ cfg.d_min
        self.pbest_position = self.position
        self.pbest_value = float('inf')
        self.velocity = np.array([0,0])
    
    def move(self):
        self.position = self.position + self.velocity

    def __str__(self):
        print(f'My position is {self.position}, my pbest is {self.pbest_position}')

class Space():
    def __init__(self, target):
        self.target = target
        self.particles = []
        self.gbest_value = float('inf')
        self.gbest_position = np.array([x * np.random.random_sample()*cfg.d_max for x in range(cfg.arguments_dimensions)])

    def print_particles(self):
        for particle in self.particles:
            particle.__str__()

    def set_pbest(self):
        for particle in self.particles:
            fitness_cadidate:float = fitness(particle.position)
            if(particle.pbest_value > fitness_cadidate):
                particle.pbest_value = fitness_cadidate
                particle.pbest_position = particle.position
            

    def set_gbest(self):
        for particle in self.particles:
            best_fitness_cadidate = fitness(particle.position)
            if(self.gbest_value > best_fitness_cadidate):
                self.gbest_value = best_fitness_cadidate
                self.gbest_position = particle.position

    def move_particles(self):
        for particle in self.particles:
            new_velocity = (cfg.w*particle.velocity) + (cfg.c1*np.random.random_sample()) * (particle.pbest_position - particle.position) + \
                            (np.random.random_sample()*cfg.c2) * (self.gbest_position - particle.position)
            particle.velocity = new_velocity
            particle.move()


# Plots

In [5]:
from numpy import exp,array,random,float32
import k3d
    
def plot_particles(particles):
    if len(particles) == 0:
        print("No particles to visualize !")
        return
    if len(particles[0].position) != 2:
        print("I can only visualize three dimensional function !")
        return
    x = list(map(lambda p: (p.position[0], p.position[1], fitness(p.position)), particles))
    plot = k3d.plot(name='points')
    plt_points = k3d.points(positions=x, point_size=0.2)
    plot += plt_points
    plt_points.shader='3d'
    plot.display()
    
# def plot_fitness(particles):
#     from matplotlib.tri import Triangulation
#     import numpy as np

#     x = list(map(lambda p: p.position[0], particles))
#     y = list(map(lambda p: p.position[1], particles))
#     z = list(map(lambda p: fitness(p.position), particles)) # evaluation of the function on the grid
#     indices = Triangulation(x,y).triangles.astype(np.float64)

#     plot = k3d.plot()
#     plt_mesh = k3d.mesh(np.vstack([x,y,z]).T, indices,
#                    color_map = k3d.colormaps.basic_color_maps.Jet,
#                    attribute=z,
#                    color_range = [-1.1,2.01])
#     plot += plt_mesh
#     plot.display()


# App

In [6]:
import numpy as np
np.set_printoptions(precision=2, floatmode='fixed', sign=' ', )
class App():
    def run(self):
        if(cfg.algorithmType == "pso"):
            search_space = Space(1)
            particles_vector = [Particle() for _ in range(cfg.n_particles)]
            search_space.particles = particles_vector
            plot_particles(search_space.particles)
            iteration = 0
            while iteration < cfg.iterations:
                search_space.set_pbest()
                search_space.set_gbest()

                if abs(search_space.gbest_value - search_space.target) <= cfg.target_error:
                    break

                search_space.move_particles()
                iteration += 1
            print("The best solution is: ", search_space.gbest_position, " in n_iterations: ", iteration)
            plot_particles(search_space.particles)
            # search_space.print_particles()
        elif(cfg.algorithmType == "ga"):
            genetic()

        else:
            print(f"Unknown algorithm type: {cfg.algorithmType}")

In [7]:
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "sphere",
    "w": 0.55,
    "c1": 0.2,
    "c2": 0.9,
    "target_error": 1e-8,
    "iterations": 200,
    "n_particles": 400,
    "arguments_dimensions": 2,
    "d_min": 200,
    "d_max": 200
}"""
cfg = json.loads(json_config, object_hook = as_config)

# Fitness functions

In [8]:
from numpy import sum, asarray_chkfinite, arange, prod, sqrt, cos, sin
from functools import reduce
def fitness(position, fr=4000):
    if cfg.fitness == "sincos":
        return sin(position[0]) + cos(position[1])
    elif cfg.fitness == "sphere":
        return reduce(lambda p,n: p + n**2, position)
    elif cfg.fitness == "rosen":
        return sum(100.0*(position[1:]-position[:-1]**2.0)**2.0 + (1-position[:-1])**2.0)
    elif cfg.fitness == "griewank":
        n = len(position)
        j = arange( 1., n+1 )
        s = sum( position**2 )
        p = prod( cos( position / sqrt(j) ))
        return s/fr - p + 1
    elif cfg.fitness == "3":
        s = 0
        for x in range(1, len(position)):
            s = s + (position[x]-x)**2
        return s

In [9]:
App().run()

Output()

The best solution is:  [ 200.00  200.00]  in n_iterations:  200


Output()